## Generate SQL database

This code will try to generate the SQL relational database for the NEB project.  We will generate the database through SQLAlchemy and sqlite

In [14]:
import sqlalchemy
import pandas as pd

In [15]:
#Declare a base
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [16]:
from sqlalchemy import Column, Integer, String, Float

In [17]:
#Declare the three tables we will be using: peaks, markers, and regions
class Peak(Base):
    __tablename__ = 'peaks'
        
    ts_data_id = Column(String, primary_key=True)
    well_id = Column(String, primary_key=True)
    samp_desc = Column(String, nullable=True)
    size = Column(Integer)
    cal_conc = Column(Float)
    assigned_conc = Column(Float, nullable=True)
    peak_mol = Column(Float)
    int_area = Column(Float)
    peak_comment = Column(String, nullable=True)
    observation = Column(String, nullable=True)
    peak_id = Column(Integer, primary_key=True)
    

In [18]:
class Marker(Base):
    __tablename__ = 'markers'
    ts_data_id = Column(String, primary_key=True)
    well_id = Column(String, primary_key=True)
    samp_desc = Column(String, nullable=True)
    size = Column(Integer)    
    cal_conc = Column(Float)
    assigned_conc = Column(Float, nullable=True)
    peak_mol = Column(Float)
    int_area = Column(Float, nullable=True)
    peak_comment = Column(String, nullable=True)
    marker_id = Column(String, primary_key=True)

In [19]:
class Region(Base):
    __tablename__ = 'regions'
    ts_data_id = Column(String, primary_key=True)
    well_id = Column(String, primary_key=True)
    samp_desc = Column(String, nullable=True)
    from_bp = Column(Integer)
    to_bp = Column(Integer)
    avg_size = Column(Integer)
    concentration = Column(Float)
    region_mol = Column(Float)
    pct_total = Column(Float)
    region_comment = Column(String, nullable=True)

In [20]:
#Declare an engine to work on
from sqlalchemy import create_engine
engine = create_engine('sqlite:///tapestation.db', echo=True)

In [21]:
Base.metadata.create_all(engine)

2020-03-20 22:31:36,541 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-03-20 22:31:36,543 INFO sqlalchemy.engine.base.Engine ()
2020-03-20 22:31:36,544 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-03-20 22:31:36,546 INFO sqlalchemy.engine.base.Engine ()
2020-03-20 22:31:36,548 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("regions")
2020-03-20 22:31:36,549 INFO sqlalchemy.engine.base.Engine ()
2020-03-20 22:31:36,551 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("regions")
2020-03-20 22:31:36,552 INFO sqlalchemy.engine.base.Engine ()
2020-03-20 22:31:36,554 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("peaks")
2020-03-20 22:31:36,555 INFO sqlalchemy.engine.base.Engine ()
2020-03-20 22:31:36,557 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("peaks")
2020-03-20 22:31:36,558 INFO sqlalchemy.engine.base.Engine ()
2020-03-20 22:31:36,55

In [22]:
#Starting a session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [23]:
session = Session()

In [24]:
import os
rootdir = '/Users/louispenafiel/Programming/Python/job_hunt/neb/ts_data/'

In [25]:
conn = engine.connect()

In [26]:
#Go through list of data directories
for name in os.listdir(rootdir):
    #Each of them ends with a 1000 and avoids callind .DS_Store
    if name.endswith("1000"):
        newdir = rootdir + name + '/'
        #instantiates the filenames
        peaktable_filename = newdir + name + '_compactPeakTable.csv'
        regiontable_filename = newdir + name + '_compactRegionTable.csv'
        
        #create pd dataframes from the filenames
        peaktable_df = pd.read_csv(peaktable_filename)
        regiontable_df = pd.read_csv(regiontable_filename)
        
        ##########################
        # GENERATING PEAKS TABLE #
        ##########################
        
        #Choose only data free from electronic ladders and synthetic markers
        peak_mask = (peaktable_df['Observations'] != 'Lower Marker') & (peaktable_df['Observations'] != 'Upper Marker')
        peak_df = peaktable_df[peak_mask].copy()
        
        #Sort the values by integrated area
        peak_df = peak_df.sort_values(by='% Integrated Area', ascending=False)
        
        #Give each peak a peak_id, depending on the size of the peak
        #Largest peak is assigned 1, second largest 2, ...
        peak_df['Peak_id'] = peak_df.groupby(['FileName','Well']).cumcount() + 1
        
        #Rename columns
        peak_df.columns = ['ts_data_id', 'well_id', 'samp_desc', 'size', 'cal_conc', 'assigned_conc', 'peak_mol', 'int_area', 'peak_comment', 'observation','peak_id']
        peak_df.to_sql('peaks', engine, index=False, if_exists='append')
        
        ############################
        # GENERATING MARKERS TABLE #
        ############################
        
        #Choose only data that correspond to synthetic markers
        marker_mask = (peaktable_df['Observations'] == 'Lower Marker') | (peaktable_df['Observations'] == 'Upper Marker')
        marker_df = peaktable_df[marker_mask].copy()
        
        marker_df.columns = ['ts_data_id', 'well_id', 'samp_desc', 'size', 'cal_conc', 'assigned_conc','peak_mol', 'int_area', 'peak_comment', 'marker_id']
        marker_df.to_sql('markers', engine, index=False, if_exists='append')

        
        ############################
        # GENERATING REGIONS TABLE #
        ############################
        
        region_df = regiontable_df.copy()

        region_df.columns = ['ts_data_id', 'well_id','samp_desc', 'from_bp', 'to_bp', 'avg_size', 'concentration', 'region_mol', 'pct_total', 'region_comment']
        region_df.to_sql('regions', engine, index=False, if_exists='append')
        
        
    else:
        continue

2020-03-20 22:31:41,961 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("peaks")
2020-03-20 22:31:41,962 INFO sqlalchemy.engine.base.Engine ()
2020-03-20 22:31:41,964 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-20 22:31:41,969 INFO sqlalchemy.engine.base.Engine INSERT INTO peaks (ts_data_id, well_id, samp_desc, size, cal_conc, assigned_conc, peak_mol, int_area, peak_comment, observation, peak_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2020-03-20 22:31:41,970 INFO sqlalchemy.engine.base.Engine (('2019-12-18 - 11.45.28.HSD1000', 'H12', None, 295, 3590.0, None, 18700, 100.0, None, None, 1), ('2019-12-18 - 11.45.28.HSD1000', 'G10', None, 293, 3290.0, None, 17300, 100.0, None, None, 1), ('2019-12-18 - 11.45.28.HSD1000', 'G6', None, 302, 7450.0, None, 37900, 100.0, None, None, 1), ('2019-12-18 - 11.45.28.HSD1000', 'F6', None, 310, 6920.0, None, 34400, 100.0, None, None, 1), ('2019-12-18 - 11.45.28.HSD1000', 'D6', None, 320, 10400.0, None, 49800, 100.0, None, None,

2020-03-20 22:31:42,064 INFO sqlalchemy.engine.base.Engine ()
2020-03-20 22:31:42,067 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-20 22:31:42,070 INFO sqlalchemy.engine.base.Engine INSERT INTO regions (ts_data_id, well_id, samp_desc, from_bp, to_bp, avg_size, concentration, region_mol, pct_total, region_comment) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2020-03-20 22:31:42,071 INFO sqlalchemy.engine.base.Engine (('2020-02-27 - 12.12.58.HSD1000', 'A1', None, 100, 1000, 338, 7510, 36700, 98.82, None), ('2020-02-27 - 12.12.58.HSD1000', 'B1', None, 100, 1000, 337, 6310, 30900, 97.89, None), ('2020-02-27 - 12.12.58.HSD1000', 'C1', None, 177, 1000, 329, 7270, 36400, 96.67, None), ('2020-02-27 - 12.12.58.HSD1000', 'D1', None, 100, 1000, 333, 5820, 28700, 98.71, None), ('2020-02-27 - 12.12.58.HSD1000', 'E1', None, 100, 1000, 323, 6020, 30400, 98.76, None), ('2020-02-27 - 12.12.58.HSD1000', 'F1', None, 100, 1000, 327, 5990, 29900, 98.45, None), ('2020-02-27 - 12.12.58.HSD1000', 'G1'

2020-03-20 22:31:42,160 INFO sqlalchemy.engine.base.Engine COMMIT
2020-03-20 22:31:42,167 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("markers")
2020-03-20 22:31:42,168 INFO sqlalchemy.engine.base.Engine ()
2020-03-20 22:31:42,171 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-20 22:31:42,175 INFO sqlalchemy.engine.base.Engine INSERT INTO markers (ts_data_id, well_id, samp_desc, size, cal_conc, assigned_conc, peak_mol, int_area, peak_comment, marker_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2020-03-20 22:31:42,176 INFO sqlalchemy.engine.base.Engine (('2020-02-03 - 11.25.42.HSD1000', 'EL1', 'Electronic Ladder', 25.0, 340.0, None, 20900.0, None, None, 'Lower Marker'), ('2020-02-03 - 11.25.42.HSD1000', 'EL1', 'Electronic Ladder', 1500.0, 250.0, 250.0, 256.0, None, None, 'Upper Marker'), ('2020-02-03 - 11.25.42.HSD1000', 'A1', 'Lot41_-DTT_1', 25.0, 355.0, None, 21800.0, None, None, 'Lower Marker'), ('2020-02-03 - 11.25.42.HSD1000', 'A1', 'Lot41_-DTT_1', 1500.0, 2

2020-03-20 22:31:42,261 INFO sqlalchemy.engine.base.Engine COMMIT
2020-03-20 22:31:42,279 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("peaks")
2020-03-20 22:31:42,281 INFO sqlalchemy.engine.base.Engine ()
2020-03-20 22:31:42,283 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-20 22:31:42,289 INFO sqlalchemy.engine.base.Engine INSERT INTO peaks (ts_data_id, well_id, samp_desc, size, cal_conc, assigned_conc, peak_mol, int_area, peak_comment, observation, peak_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2020-03-20 22:31:42,290 INFO sqlalchemy.engine.base.Engine (('2020-02-11 - 10.01.14.HSD1000', 'H7', None, 321, 1470.0, None, 7060.0, 95.22, None, None, 1), ('2020-02-11 - 10.01.14.HSD1000', 'F4', None, 154, 836.0, None, 8340.0, 95.16, None, None, 1), ('2020-02-11 - 10.01.14.HSD1000', 'G9', None, 314, 731.0, None, 3580.0, 93.8, None, None, 1), ('2020-02-11 - 10.01.14.HSD1000', 'C7', None, 313, 832.0, None, 4090.0, 93.05, None, None, 1), ('2020-02-11 - 10.01.14.HSD

In [57]:
session.new

IdentitySet([])

In [27]:
session.commit()

In [60]:
for instance in session.query(Peak):
    print(instance.ts_data_id, instance.well_id, instance.peak_id)

2020-03-19 02:56:53,008 INFO sqlalchemy.engine.base.Engine SELECT peaks.ts_data_id AS peaks_ts_data_id, peaks.well_id AS peaks_well_id, peaks.size AS peaks_size, peaks.cal_conc AS peaks_cal_conc, peaks.peak_mol AS peaks_peak_mol, peaks.int_area AS peaks_int_area, peaks.peak_id AS peaks_peak_id 
FROM peaks
2020-03-19 02:56:53,009 INFO sqlalchemy.engine.base.Engine ()
(u'2019-12-18 - 11.45.28.HSD1000', u'A1', 1)
(u'2019-12-18 - 11.45.28.HSD1000', u'G7', 1)
(u'2019-12-18 - 11.45.28.HSD1000', u'A9', 1)
(u'2019-12-18 - 11.45.28.HSD1000', u'G8', 1)
(u'2019-12-18 - 11.45.28.HSD1000', u'F8', 1)
(u'2019-12-18 - 11.45.28.HSD1000', u'E8', 1)
(u'2019-12-18 - 11.45.28.HSD1000', u'D8', 1)
(u'2019-12-18 - 11.45.28.HSD1000', u'C8', 1)
(u'2019-12-18 - 11.45.28.HSD1000', u'B8', 1)
(u'2019-12-18 - 11.45.28.HSD1000', u'A8', 1)
(u'2019-12-18 - 11.45.28.HSD1000', u'B1', 1)
(u'2019-12-18 - 11.45.28.HSD1000', u'B6', 1)
(u'2019-12-18 - 11.45.28.HSD1000', u'D7', 1)
(u'2019-12-18 - 11.45.28.HSD1000', u'C7', 1)
(